In [ ]:
!pip install "opencv-python-headless<4.3"
!pip install cellpose

In [ ]:
!nvcc --version
!nvidia-smi

import os, shutil
import numpy as np
import matplotlib.pyplot as plt
from cellpose import core, utils, io, models, metrics
from glob import glob

use_GPU = core.use_gpu()
yn = ['NO', 'YES']
print(f'>>> GPU activated? {yn[use_GPU]}')

In [ ]:
import numpy as np
import time, os, sys
from urllib.parse import urlparse
import matplotlib.pyplot as plt
import matplotlib as mpl
%matplotlib inline
mpl.rcParams['figure.dpi'] = 200
from cellpose import utils, io, transforms
import random
import PIL

In [ ]:
data_dir = '/path/to/dir'

In [ ]:
all_files = []
import os
for root, dirs, files in os.walk(data_dir):
  for fn in files:
    if fn.endswith(".tif"):
      all_files.append(os.path.join(root, fn))


In [ ]:
# @title Control

all_files_copy = [x for x in all_files if 'DAPI' in x and 'PHAL' in x and 'Control' in x]
random.shuffle(all_files_copy)
imgs = []
for iex, fn in enumerate(all_files_copy[:6]):
  img = PIL.Image.open(fn)
  img = np.array(img)
  # resize to 400x400
  if img.shape[0] > 400 or img.shape[1] > 400:
    img = transforms.resize_image(img, Lx=400, Ly=400)
  # Change to C, X, Y
  if img.shape[2] == 3:
    img = img.transpose(2,0,1)
  imgs.append(img)



In [ ]:
from cellpose import io

io.logger_setup() # run this to get printing of progress

# DEFINE CELLPOSE MODEL WITH SIZE
# model_type='cyto3' or model_type='nuclei'
model = models.Cellpose(gpu=True, model_type="cyto3")

# define CHANNELS to run segementation on
# grayscale=0, R=1, G=2, B=3
# channels = [cytoplasm, nucleus]
# if NUCLEUS channel does not exist, set the second channel to 0
# channels = [0,0]
# IF ALL YOUR IMAGES ARE THE SAME TYPE, you can give a list with 2 elements
# channels = [0,0] # IF YOU HAVE GRAYSCALE
# channels = [2,3] # IF YOU HAVE G=cytoplasm and B=nucleus
# channels = [2,1] # IF YOU HAVE G=cytoplasm and R=nucleus
# OR if you have different types of channels in each image
# channels = [[2,3], [0,0], [0,0]]

# FOR A CUSTOM MODEL OR OTHER BUILT-IN MODEL:
# model = models.CellposeModel(gpu=True, model_type="livecell_cp3")

masks_pred, flows, styles, diams = model.eval(imgs, diameter=0, channels=[2,3],
                                              niter=2000) # using more iterations for bacteria


In [ ]:
import random
import PIL

plt.figure(figsize=(12,6))
ly = 400
for iex in range(len(imgs)):
  img = imgs[iex].squeeze().copy()
  img = img.transpose(1,2,0)
  ax = plt.subplot(3, 8, (iex%3)*8 + (iex//3) +1)
  ax.imshow(img)
  ax.set_ylim([0, min(400, img.shape[0])])
  ax.set_xlim([0, min(400, img.shape[1])])


  # GROUND-TRUTH = PURPLE
  # PREDICTED = YELLOW
  outlines_pred = utils.outlines_list(masks_pred[iex])
  for o in outlines_pred:
      plt.plot(o[:,0], o[:,1], color=[1,1,0.3], lw=0.75, ls="--")
  plt.axis('off')

plt.tight_layout()
plt.show()

In [ ]:
# @title Treated

all_files_2 = [x for x in all_files if 'DAPI' in x and 'PHAL' in x and not 'Control' in x]
random.shuffle(all_files_2)
imgs2 = []
for iex, fn in enumerate(all_files_2[:6]):
  img = PIL.Image.open(fn)
  img = np.array(img)
  # resize to 400x400
  if img.shape[0] > 400 or img.shape[1] > 400:
    img = transforms.resize_image(img, Lx=400, Ly=400)
  # Change to C, X, Y
  if img.shape[2] == 3:
    img = img.transpose(2,0,1)
  imgs2.append(img)



In [ ]:
# define CHANNELS to run segementation on
# grayscale=0, R=1, G=2, B=3
# channels = [cytoplasm, nucleus]
# if NUCLEUS channel does not exist, set the second channel to 0
# channels = [0,0]
# IF ALL YOUR IMAGES ARE THE SAME TYPE, you can give a list with 2 elements
# channels = [0,0] # IF YOU HAVE GRAYSCALE
# channels = [2,3] # IF YOU HAVE G=cytoplasm and B=nucleus
# channels = [2,1] # IF YOU HAVE G=cytoplasm and R=nucleus
# OR if you have different types of channels in each image
# channels = [[2,3], [0,0], [0,0]]

# FOR A CUSTOM MODEL OR OTHER BUILT-IN MODEL:
# model = models.CellposeModel(gpu=True, model_type="livecell_cp3")

masks_pred, flows, styles, diams = model.eval(imgs2, diameter=0, channels=[2,3],
                                              niter=2000) # using more iterations for bacteria


In [ ]:
plt.figure(figsize=(12,6))
ly = 400
for iex in range(len(imgs2)):
  img = imgs2[iex].squeeze().copy()
  img = img.transpose(1,2,0)
  ax = plt.subplot(3, 8, (iex%3)*8 + (iex//3) +1)
  ax.imshow(img)
  ax.set_ylim([0, min(400, img.shape[0])])
  ax.set_xlim([0, min(400, img.shape[1])])


  # GROUND-TRUTH = PURPLE
  # PREDICTED = YELLOW
  outlines_pred = utils.outlines_list(masks_pred[iex])
  for o in outlines_pred:
      plt.plot(o[:,0], o[:,1], color=[1,1,0.3], lw=0.75, ls="--")
  plt.axis('off')

plt.tight_layout()
plt.show()